# Dynamic
## Dynamic Prompt

In [1]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")

In [2]:
from langchain.chat_models import init_chat_model

model = init_chat_model("openai:gpt-5")

In [3]:
from langchain_core.tools import tool


@tool
def execute_sql(query: str) -> str:
    """Execute a SQLite command and return results."""
    try:
        return db.run(query)
    except Exception as e:
        return f"Error: {e}"

In [ ]:
from dataclasses import dataclass


@dataclass
class RuntimeContext:
    is_employee: bool

In [5]:
SYSTEM_PROMPT_TEMPLATE = """You are a careful SQLite analyst.

Rules:
- Think step-by-step.
- When you need data, call the tool `execute_sql` with ONE SELECT query.
- Read-only only; no INSERT/UPDATE/DELETE/ALTER/DROP/CREATE/REPLACE/TRUNCATE.
- Limit to 5 rows unless the user explicitly asks otherwise.
{table_limits}
- If the tool returns 'Error:', revise the SQL and try again.
- Prefer explicit column lists; avoid SELECT *.
"""

In [ ]:
from langchain.agents.middleware.types import AgentState, ModelRequest, dynamic_prompt
from langgraph.runtime import Runtime


@dynamic_prompt
def dynamic_system_prompt(
    request: ModelRequest, state: AgentState, runtime: Runtime[RuntimeContext]
) -> str:
    if not runtime.context.is_employee:
        table_limits = "- Limit access to these tables: Album, Artist, Genre, Playlist, PlaylistTrack, Track."
    else:
        table_limits = ""

    return SYSTEM_PROMPT_TEMPLATE.format(table_limits=table_limits)

In [7]:
from langchain.agents import create_agent

agent = create_agent(
    model="openai:gpt-5",
    tools=[execute_sql],
    middleware=[dynamic_system_prompt],
    context_schema=RuntimeContext,
)

In [ ]:
question = "What is the most costly purchase by Frank Harris?"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    context=RuntimeContext(is_employee=False),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

What is the most costly purchase by Frank Harris?
================================== Ai Message ==================================

I can’t determine purchases with the tables I’m allowed to query. “Most costly purchase by Frank Harris” would require the Customer, Invoice, and InvoiceLine tables (to find his largest transaction or most expensive item).

Could you confirm:
- Do you mean the customer named “Frank Harris”? If so, please allow access to Customer, Invoice, and InvoiceLine (and specify whether you want his largest single invoice total or the most expensive item he bought).
- Or did you mean something else (e.g., an artist named Frank Harris and the most expensive track/album)?


In [ ]:
question = "What is the most costly purchase by Frank Harris?"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    context=RuntimeContext(is_employee=True),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

What is the most costly purchase by Frank Harris?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (call_wkYuOVn6LPKsd0kvfsFtEI9b)
 Call ID: call_wkYuOVn6LPKsd0kvfsFtEI9b
  Args:
    query: SELECT name FROM sqlite_master WHERE type='table' ORDER BY name LIMIT 5;
================================= Tool Message =================================
Name: execute_sql

[('Album',), ('Artist',), ('Customer',), ('Employee',), ('Genre',)]
================================== Ai Message ==================================
Tool Calls:
  execute_sql (call_ORHys3QG09Z9ae5O7zF2To9I)
 Call ID: call_ORHys3QG09Z9ae5O7zF2To9I
  Args:
    query: SELECT i.InvoiceId, i.InvoiceDate, i.Total
FROM Invoice i
JOIN Customer c ON i.CustomerId = c.CustomerId
WHERE c.FirstName = 'Frank' AND c.LastName = 'Harris'
ORDER BY i.Total DESC, i.InvoiceDate DESC
LIMIT 5;
=====================